<a href="https://colab.research.google.com/github/P2Enjoy/ia-web3.fr/blob/main/Copie_de_SpeedWatch_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
#@title Install all requirements (run once)

# System
import locale, os
locale.getpreferredencoding = lambda: "UTF-8"
!pip install --upgrade transformers==4.31.0 torch accelerate==0.21.0 bitsandbytes==0.41.0 tensorflow==2.13.0 onnxruntime-gpu==1.15.1
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# Application
!pip install deepmultilingualpunctuation
!pip install spacy
!python -m spacy download en_core_web_md
!pip install scikit-learn
!pip install pytorch-crf==0.7.2 youtube_transcript_api==0.6.1 fastpunct==2.0.2 nnsplit==0.5.9.post0 tiktoken==0.4.0
import nltk
nltk.download('punkt')
!git clone https://github.com/xashru/punctuation-restoration.git
!gdown https://drive.google.com/u/0/uc?id=17BPcnHVhpQlsOTC8LEayIFFJ7WkL00cr&export=download

2023-07-31 08:37:28.376783: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 19.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
fatal: destination path 'punctuation-restoration' already exists and is not an empty directory.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Downloading...
From: https://drive.google.com/u/0/uc?id=17BPcnHVhpQlsOTC8LEayIFFJ7WkL00cr
To: /content/roberta-large-en.pt
100% 1.49G/1.49G [00:16<00:00, 92.6MB/s]


# Youtube processing

In [22]:
#@title Put the link of the video to summarize:
video_link = "https://www.youtube.com/watch?v=jQL0ZeHtXFc" #@param {type:"string"}

In [23]:
#@title Extract the youtube video transcript

def YTVideoToText(video_link):
  # installing & importing libraries
  from youtube_transcript_api import YouTubeTranscriptApi

  # fetching video transcript
  video_id = video_link.split("=")[1]
  transcript = YouTubeTranscriptApi.get_transcript(video_id)

  # iterating throughout and adding all text together (recovering ponctuations)
  flat_transcript = ""
  for tu in transcript:
      flat_transcript += ' ' + tu['text']

  return flat_transcript

video_full_text = YTVideoToText(video_link)

In [24]:
#@title Sentence splitter
max_tokens_per_sentence = 416 #@param {type:"slider", min:64, max:8192, step:32}

import tiktoken
from nnsplit import NNSplit
splitter = NNSplit.load("en")
encoding = tiktoken.encoding_for_model("gpt2")

# returns `Split` objects
video_sentences = []
current_sentence = ""
for sentence in (splitter.split([video_full_text])[0]):
  if (len(encoding.encode(f"{current_sentence}")) < max_tokens_per_sentence):
    current_sentence += f"{sentence}"
  else:
    video_sentences.append(current_sentence)
    current_sentence = ""


In [25]:
#@title Extract the sentences
import spacy

def process_text(full_text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(full_text)
    return [f'{sentence}' for sentence in doc.sents]

# Utilisation de la fonction
video_sentences = process_text(video_full_text)



In [26]:
#@title restore the ponctuation

from deepmultilingualpunctuation import PunctuationModel

model = PunctuationModel()
text = video_full_text
video_result = model.restore_punctuation(text)

print(video_result)


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:169: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="none"` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


hello Community, welcome to the most watched part of how to instruct, fine tune your large language model. after the amazing success of part 1 and part 2, now part three. so here we are. now we have here a look at the prompt, Stanford alpaca prompt. so we are taking you exactly this prompt. remember, this is step one, like I showed you. now we do it one by one. so we say, yeah, let's execute this. have a look at it. so we say, okay, you're asked to come up with 20 diverse task instruction. now create Step 1, 20 in structure and, as I told you, there are some requirements: try not to repeat the verb. the language should be diverse. the type of instruction should be diverse. the language- chat, GPT or GPT language suit model- should be able to complete the instruction. it will be in English. the instructions should be one to two sentences long. you should generate an appropriate input to the instruction. not all reinstructions require input. that's exactly as I showed you in my last vide

In [32]:
#@title build paragraphs
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering

# Charger le modèle SpaCy en anglais
nlp = spacy.load("en_core_web_md")

# Exemple de texte avec des phrases
text = video_result

# Fonction pour vectoriser les phrases en utilisant SpaCy
def vectorize_sentences(sentences):
    return [nlp(sentence).vector for sentence in sentences]

# Diviser le texte en phrases
sentences = [sentence.text for sentence in nlp(text).sents]

# Vectoriser les phrases
sentence_vectors = vectorize_sentences(sentences)

# Calculer la matrice de similarité du cosinus
similarity_matrix = cosine_similarity(sentence_vectors)

# Appliquer un algorithme de clustering (ici, Agglomerative Clustering)
cluster = AgglomerativeClustering(n_clusters=None, distance_threshold=0.4, affinity='precomputed', linkage='average')
clusters = cluster.fit_predict(1 - similarity_matrix)

# Regrouper les phrases en paragraphes en fonction du clustering
paragraphs = {}
for i, cluster_id in enumerate(clusters):
    if cluster_id not in paragraphs:
        paragraphs[cluster_id] = []
    paragraphs[cluster_id].append(sentences[i])

# Former les paragraphes en concaténant les phrases
paragraphs = [' '.join(paragraph) for paragraph in paragraphs.values()]

# Afficher les paragraphes résultants
for i, paragraph in enumerate(paragraphs):
    print(paragraph)



hello Community, welcome to the most watched part of how to instruct, fine tune your large language model. after the amazing success of part 1 and part 2, now part three. so here we are. now we have here a look at the prompt, Stanford alpaca prompt. so we are taking you exactly this prompt. remember, this is step one, like I showed you. now we do it one by one. so we say, yeah, let's execute this. have a look at it. so we say, okay, you're asked to come up with 20 diverse task instruction. now create Step 1, 20 in structure and, as I told you, there are some requirements: try not to repeat the verb. the language should be diverse. the type of instruction should be diverse. the language- chat, GPT or GPT language suit model- should be able to complete the instruction. it will be in English. the instructions should be one to two sentences long. you should generate an appropriate input to the instruction. not all reinstructions require input. that's exactly as I showed you in my last vide

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


In [ ]:
#@title Restore video transcript punctuation
spell_check = True #@param {type:"boolean"}

def FlatTextToPunct(txt):
  # installing & importing libraries
  from fastpunct import FastPunct

  # The default language is 'english'
  fastpunct = FastPunct()

  return fastpunct.punct(txt, correct= spell_check)

#video_restored_text = FlatTextToPunct(video_sentences)



In [ ]:
with open("video_full_text", 'w', encoding='utf-8') as f:
  f.write(video_full_text)

In [ ]:
%%shell

python punctuation-restoration/src/inference.py --pretrained-model=roberta-large --weight-path=roberta-large-en.pt --language=en
--in-file=video_full_text --out-file=video_restored_text

In [ ]:
with open("video_restored_text", 'r', encoding='utf-8') as f:
    video_restored_text = f.read()

# Summarization

In [ ]:
#@title Load the model into memory
from transformers import pipeline

model_id = "facebook/bart-large-cnn" #@param {type:"string"}
summarizeryt = pipeline('summarization', model= model_id, device=0)

In [ ]:
#@title Join sentences in paragraphs of less the specified token size
summarize_token_width = 500 #@param {type:"slider", min:250, max:2048, step:250}

In [ ]:

def do_sum(sentences):
  # summarize text
  summarized_text = ''
  for sentence in sentences:
      out = summarizeryt(sentence, min_length=30, do_sample=False)
      out = out[0]
      out = out['summary_text']
      summarized_text.append(' ').append(out)

  # returning summary
  return summarized_text;

In [ ]:
#@title Show the transcript of the video
video_full_text

In [ ]:
#@title Show the summarized text of the video
sum_text